<a href="https://colab.research.google.com/github/DavidAntonio7/data_frame_brasileirao/blob/main/montando_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import seaborn as sns
from datetime import datetime

In [25]:
df = pd.read_csv('brasileirao-jogos.csv',sep = ';')
df.head()

,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor
0,1,29/03/2003,16h00,Sábado,Athlético-PR,Grêmio,Athlético-PR,Arena da Baixada,2,0,PR,RS,PR
1,1,29/03/2003,16h00,Sábado,Guarani,Vasco,Guarani,Brinco de Ouro,4,2,SP,RJ,SP
2,1,30/03/2003,16h00,Domingo,Fortaleza,Bahia,-,Castelão,0,0,CE,BA,-
3,1,30/03/2003,16h00,Domingo,Flamengo,Coritiba,-,Maracanã,1,1,RJ,PR,-
4,1,30/03/2003,16h00,Domingo,Vitória,Figueirense,-,Barradão,1,1,ES,SC,-


In [26]:
df.drop(columns=['Horário','Dia','Arena','Estado Mandante','Estado Visitante', 'Estado Vencedor'],inplace=True)

In [27]:
df1 = df.copy()

In [28]:
##tratando as datas para que apareca somente o ano da temporada
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y').dt.year

dados faltantes

In [29]:
colunas = ['Rodada', 'Data', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar']
jogos = [
    [2,2008,'Cruzeiro','Botafogo-RJ','Cruzeiro',1,0],
    [2,2008,'Sport','Vitória','-',0,0],
    [2,2008,'Vasco','Portuguesa','-',3,1]
        ]
faltantes_2008_2 = pd.DataFrame(jogos,columns=colunas)
df = pd.concat([df,faltantes_2008_2])

In [30]:
df_mandante = df.copy()
df_visitante = df.copy()

In [31]:
df_mandante['Time'] = df_mandante.Mandante
df_mandante['Mando_de_campo'] = 1
df_visitante['Time'] = df_visitante.Visitante
df_visitante['Mando_de_campo'] = -1

In [32]:
df_visitante.index = range(df_visitante.shape[0],df_visitante.shape[0]+df_visitante.shape[0])
df_times = pd.concat([df_visitante,df_mandante])
df_times.shape

(14538, 9)

In [33]:
df_times1 = df_times.copy()

In [34]:
df_times = df_times1.copy()
df_times.rename(columns={'Data' : 'Temporada'},inplace=True)
df_times.head(1)

,Rodada,Temporada,Mandante,Visitante,Vencedor,Mandante Placar,Visitante Placar,Time,Mando_de_campo
7269,1,2003,Athlético-PR,Grêmio,Athlético-PR,2,0,Grêmio,-1


In [35]:
df_times.columns

Index(['Rodada', 'Temporada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Time', 'Mando_de_campo'],
      dtype='object')

In [36]:
df_times = df_times[['Time','Temporada','Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar','Mando_de_campo']]

##funcao para calcular a tabela a partir dos resultados

In [37]:
def calculaTabela(x):    
    pontos = 0;vitorias = 0;derrotas = 0;empates = 0;gols_pro = 0;gols_contra = 0;vitorias_consecutivas = 0;
    derrotas_consecutivas = 0;jogos_marcando = 0;jogos_sem_sofrer_gols = 0;vencedor = 0
    for i in range(0,x.shape[0]):
        ##se o time for visitante
        if x.iloc[i].Time == x.iloc[i].Visitante:
            ##pontos - vitorias - derrotas - empates - vitorias concecutivas - derrotas concecutivas
            if x.iloc[i].Visitante == x.iloc[i].Vencedor: #vencedor
                vitorias += 1;vitorias_consecutivas += 1; derrotas_consecutivas = 0; pontos+=3
            elif x.iloc[i].Mandante == x.iloc[i].Vencedor:
                derrotas+= 1;vitorias_consecutivas = 0; derrotas_consecutivas +=1; pontos+=0
            else:
                empates+= 1;vitorias_consecutivas = 0; derrotas_consecutivas = 0; pontos+=1
            ##metricas de gols 
            gols_pro += x.iloc[i]['Visitante Placar']; gols_contra += x.iloc[i]['Mandante Placar'];
            if x.iloc[i]['Visitante Placar'] > 0:
                if jogos_marcando >= 0:
                    jogos_marcando+=1
                else:
                    jogos_marcando = 1
            elif x.iloc[i]['Visitante Placar'] <= 0:
                if jogos_marcando >= 0:
                    jogos_marcando = -1
                else:
                    jogos_marcando += -1
            if x.iloc[i]['Mandante Placar'] > 0:
                if jogos_sem_sofrer_gols >= 0:
                    jogos_sem_sofrer_gols = -1
                else:
                    jogos_sem_sofrer_gols -= 1
            elif x.iloc[i]['Mandante Placar'] == 0:
                if jogos_sem_sofrer_gols >= 0:
                    jogos_sem_sofrer_gols = +1
                else:
                    jogos_sem_sofrer_gols += 1
        ##se o time for mandante
        if x.iloc[i].Time == x.iloc[i].Mandante:
            ##pontos - vitorias - derrotas - empates - vitorias concecutivas - derrotas concecutivas
            if x.iloc[i].Mandante == x.iloc[i].Vencedor: #vencedor
                vitorias += 1;vitorias_consecutivas += 1; derrotas_consecutivas = 0; pontos+=3
            elif x.iloc[i].Visitante == x.iloc[i].Vencedor:
                derrotas+= 1;vitorias_consecutivas = 0; derrotas_consecutivas +=1; pontos+=0
            else:
                empates+= 1;vitorias_consecutivas = 0; derrotas_consecutivas = 0; pontos+=1
            ##metricas de gols 
            gols_pro += x.iloc[i]['Mandante Placar']; gols_contra += x.iloc[i]['Visitante Placar'];
            if x.iloc[i]['Mandante Placar'] > 0:
                if jogos_marcando >= 0:
                    jogos_marcando+=1
                else:
                    jogos_marcando = 1
            elif x.iloc[i]['Mandante Placar'] <= 0:
                if jogos_marcando >= 0:
                    jogos_marcando = -1
                else:
                    jogos_marcando += -1
            if x.iloc[i]['Visitante Placar'] > 0:
                if jogos_sem_sofrer_gols >= 0:
                    jogos_sem_sofrer_gols = -1
                else:
                    jogos_sem_sofrer_gols -= 1
            elif x.iloc[i]['Visitante Placar'] == 0:
                if jogos_sem_sofrer_gols >= 0:
                    jogos_sem_sofrer_gols = +1
                else:
                    jogos_sem_sofrer_gols = 1
        x.iloc[i,9:] = [pontos,vitorias,derrotas,empates,gols_pro,gols_contra,(gols_pro-gols_contra),vitorias_consecutivas,derrotas_consecutivas,
    jogos_marcando,jogos_sem_sofrer_gols]
    return x
                

In [38]:
#ordenando as colunas
df_times = df_times[['Time','Temporada','Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar','Mando_de_campo']]

In [39]:
#criando novas colunas
df_times[['Pontos','Vitoria','Derrota','Empate','Gols_pro','Gols_contra','Saldo_de_gols',
   'Vitorias_consecutivas','Derrotas_consecutivas','Jogos_marcando','Jogos_sem_sofrer_gols']] = 0

In [40]:
#ordenando a base
df_times.sort_values(['Temporada','Rodada'],inplace=True)

In [41]:
#removendo as temporadas com mais de 20 times e outras com erros 
df_times['Temporada'] = df_times['Temporada'].astype(int)
df_times = df_times[df_times['Temporada'] > 2007]

In [42]:
df_times2 = df_times.copy()

##consertando valores escritos em maiuscula

In [43]:
df_times.Time = df_times.Time.str.lower()
df_times.Mandante = df_times.Mandante.str.lower()
df_times.Visitante = df_times.Visitante.str.lower()
df_times.Vencedor = df_times.Vencedor.str.lower()


In [44]:
#conferindo o numero de clubes por temporada
for temp in range(2008,2021):
    print(temp ,df_times[df_times.Temporada == temp].Time.nunique(),df_times[df_times.Temporada == temp].Mandante.nunique(),
          df_times[df_times.Temporada == temp].Visitante.nunique())

2008 20 20 20
2009 20 20 20
2010 20 20 20
2011 20 20 20
2012 20 20 20
2013 20 20 20
2014 20 20 20
2015 20 20 20
2016 20 20 20
2017 20 20 20
2018 20 20 20
2019 20 20 20
2020 20 20 20


Calculando a tabela para todos times em todas temporadas

In [45]:
for temporada in range(2008, 2022):
    for time in df_times[df_times['Temporada'] == temporada]['Time'].unique():
        selecao = df_times[(df_times['Temporada'] == temporada) & (df_times['Time'] == time)].copy()
        resultado = calculaTabela(selecao)
        df_times.loc[(df_times['Temporada'] == temporada) & (df_times['Time'] == time)] = resultado

as temporadas de 2021 e 2020 estao faltando dados -> romovendo-as do df

In [46]:
df_times = df_times.query('Temporada < 2020')

## tratando a posicao na tabela

In [47]:
#ordenando o df segundo os criterios do brasileirao
df_times.sort_values(['Temporada','Rodada','Pontos','Vitoria','Saldo_de_gols'],ascending=[True,True,False,False,False],inplace=True)

<ipython-input-47-e4f4b8afba5c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_times.sort_values(['Temporada','Rodada','Pontos','Vitoria','Saldo_de_gols'],ascending=[True,True,False,False,False],inplace=True)


In [48]:
df_times.query('Temporada == 2010').tail(20)

,Time,Temporada,Rodada,Mandante,Visitante,Vencedor,Mandante Placar,Visitante Placar,Mando_de_campo,Pontos,Vitoria,Derrota,Empate,Gols_pro,Gols_contra,Saldo_de_gols,Vitorias_consecutivas,Derrotas_consecutivas,Jogos_marcando,Jogos_sem_sofrer_gols
3462,fluminense,2010,38,fluminense,guarani,fluminense,1,0,1,71,20,7,11,62,36,26,3,0,5,1
3461,cruzeiro,2010,38,cruzeiro,palmeiras,cruzeiro,2,1,1,69,20,9,9,53,38,15,3,0,3,-5
10727,corinthians,2010,38,goiás,corinthians,-,1,1,-1,68,19,8,11,65,41,24,0,0,8,-1
3463,grêmio,2010,38,grêmio,botafogo-rj,grêmio,3,0,1,63,17,9,12,68,43,25,3,0,3,1
3460,athlético-pr,2010,38,athlético-pr,avaí,athlético-pr,1,0,1,60,17,12,9,43,45,-2,1,0,9,1
10732,botafogo-rj,2010,38,grêmio,botafogo-rj,grêmio,3,0,-1,59,14,7,17,54,42,12,0,1,-1,-4
10725,internacional,2010,38,grêmio prudente,internacional,internacional,0,3,-1,58,16,12,10,48,41,7,1,0,5,-3
3459,santos,2010,38,santos,flamengo,-,0,0,1,56,15,12,11,63,50,13,0,0,-1,1
3464,são paulo,2010,38,são paulo,atlético-mg,são paulo,4,0,1,55,15,13,10,54,54,0,1,0,4,1
10730,palmeiras,2010,38,cruzeiro,palmeiras,cruzeiro,2,1,-1,50,12,12,14,42,43,-1,0,4,2,-4


In [49]:
df_times.columns

Index(['Time', 'Temporada', 'Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Mando_de_campo', 'Pontos',
       'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols'],
      dtype='object')

In [52]:
df_times['Posicao'] = 0

In [53]:
#organizando
df_times = df_times[['Time', 'Temporada', 'Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Mando_de_campo',  'Posicao','Pontos',
       'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols']]

In [54]:
df_times1 = df_times.copy()

In [55]:
# conferindo se o numeros estao correto 
df_times.groupby('Temporada').Time.count()

Temporada
2008    760
2009    760
2010    760
2011    760
2012    760
2013    760
2014    760
2015    760
2016    760
2017    760
2018    760
2019    760
Name: Time, dtype: int64

In [56]:
for temp,rodada in zip (df_times.Temporada.unique(), range(1,39)):
    df_times.loc[(df_times.Temporada == temp) & (df_times.Rodada == 38), 'Posicao'] = range(1, 21)

In [57]:
df_times[(df_times.Temporada == 2011) & (df_times.Rodada == 38)].Posicao

3839      1
3837      2
11107     3
11106     4
3844      5
3845      6
11105     7
11110     8
3838      9
11114    10
11108    11
11113    12
3840     13
3842     14
11112    15
3843     16
3841     17
11111    18
11109    19
3836     20
Name: Posicao, dtype: int64

In [58]:
#arrumando os valores dos index
df_times.index = range(0,df_times.shape[0])

fazendo uma coluna adversario

In [59]:
Adversario = []
for i in df_times.index:
    if df_times.iloc[i].Time == df_times.iloc[i].Mandante:
        Adversario.append(df_times.iloc[i].Visitante)
    else:
        Adversario.append(df_times.iloc[i].Mandante)

In [60]:
df_times['Adversario'] = Adversario

<ipython-input-60-083bb01ac981>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_times['Adversario'] = Adversario


In [61]:
df_times[['Time','Adversario']].head()

,Time,Adversario
0,cruzeiro,vitória
1,botafogo-rj,sport
2,coritiba,palmeiras
3,flamengo,santos
4,grêmio,são paulo


In [62]:
df_times.columns

Index(['Time', 'Temporada', 'Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Mando_de_campo', 'Posicao',
       'Pontos', 'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols', 'Adversario'],
      dtype='object')

In [63]:
#organizando e removendo as colunas Mandante e Visitante que nao sao mais necessarias
df_times = df_times[
    ['Time', 'Temporada', 'Rodada', 'Adversario', 'Vencedor','Mando_de_campo',
       'Mandante Placar', 'Visitante Placar',  'Posicao',
       'Pontos', 'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols']
]

In [64]:
df_times.head()

,Time,Temporada,Rodada,Adversario,Vencedor,Mando_de_campo,Mandante Placar,Visitante Placar,Posicao,Pontos,Vitoria,Derrota,Empate,Gols_pro,Gols_contra,Saldo_de_gols,Vitorias_consecutivas,Derrotas_consecutivas,Jogos_marcando,Jogos_sem_sofrer_gols
0,cruzeiro,2008,1,vitória,cruzeiro,-1,0,2,0,3,1,0,0,2,0,2,1,0,1,1
1,botafogo-rj,2008,1,sport,botafogo-rj,1,2,0,0,3,1,0,0,2,0,2,1,0,1,1
2,coritiba,2008,1,palmeiras,coritiba,1,2,0,0,3,1,0,0,2,0,2,1,0,1,1
3,flamengo,2008,1,santos,flamengo,1,3,1,0,3,1,0,0,3,1,2,1,0,1,-1
4,grêmio,2008,1,são paulo,grêmio,-1,0,1,0,3,1,0,0,1,0,1,1,0,1,1


exportando 

In [65]:
df_times.to_csv('df_brasileirao_2008_2019.csv',index = False)